In [ ]:
##导入必要的库
# coding=UTF-8
import xlrd
import xlwt
from xlutils.copy import copy
import requests
import time
import hashlib
import json

In [ ]:
##用于写入数据
def write_excel_xls(path, sheet_name, value):
    index = len(value)  # 获取需要写入数据的行数
    workbook = xlwt.Workbook()  # 新建一个工作簿
    sheet = workbook.add_sheet(sheet_name)  # 在工作簿中新建一个表格
    for i in range(0, index):
        for j in range(0, len(value[i])):
            sheet.write(i, j, value[i][j])  # 像表格中写入数据（对应的行和列）
    workbook.save(path)  # 保存工作簿
    print("xls格式表格写入数据成功！")

In [ ]:
##用于增补数据
def write_excel_xls_append(path, value):
    index = len(value)  # 获取需要写入数据的行数
    workbook = xlrd.open_workbook(path)  # 打开工作簿
    sheets = workbook.sheet_names()  # 获取工作簿中的所有表格
    worksheet = workbook.sheet_by_name(sheets[0])  # 获取工作簿中所有表格中的的第一个表格
    rows_old = worksheet.nrows  # 获取表格中已存在的数据的行数
    new_workbook = copy(workbook)  # 将xlrd对象拷贝转化为xlwt对象
    new_worksheet = new_workbook.get_sheet(0)  # 获取转化后工作簿中的第一个表格
    for i in range(0, index):
        for j in range(0, len(value[i])):
            new_worksheet.write(i+rows_old, j, value[i][j])  # 追加写入数据，注意是从i+rows_old行开始写入
    new_workbook.save(path)  # 保存工作簿

In [ ]:
#  请求参数
appkey = ""
seckey = ""
encode = 'utf-8'

# Http请求头设置
timespan = str(int(time.time()))
token = appkey + timespan + seckey;
hl = hashlib.md5()
hl.update(token.encode(encoding=encode))
token = hl.hexdigest().upper();
print('MD5加密后为 :' + token)
headers = {'Token': token, 'Timespan': timespan}

In [ ]:
## 批量操作准备工作
path = '登记号名单.xls'
book_name_xls = '数据写入.xls' 
sheet_name_xls = '抓取数据写入' 
value_title = [[ "登记号", "版本号","软著名"],]
write_excel_xls(book_name_xls, sheet_name_xls, value_title)

# 使用前务必检查名单文件

In [ ]:
workbook = xlrd.open_workbook(path)  # 打开工作簿
sheets = workbook.sheet_names()  # 获取工作簿中的所有表格
worksheet = workbook.sheet_by_name(sheets[0])  # 获取工作簿中所有表格中的的第一个表格
for i in range(1,worksheet.nrows):
    
    # Http请求头设置
    timespan = str(int(time.time()))
    token = appkey + timespan + seckey;
    hl = hashlib.md5()
    hl.update(token.encode(encoding=encode))
    token = hl.hexdigest().upper();
    #print('MD5加密后为 ：' + token)
    headers = {'Token': token, 'Timespan': timespan}
    
    registeNo = worksheet.cell_value(i, 1) # 逐行逐列读取数据
    
    url = " http://api.qichacha.com/CopyRight/GetSoftwareCr"+"?key="+ appkey+"&registeNo="+registeNo

    response = requests.get(url, headers=headers)
    print(url)
    ##结果处理  
    m=json.loads(response.text)

    if m["Result"] == None:
        print(m)
        value = [[registeNo,"无数据"],]
        write_excel_xls_append(book_name_xls, value)
        print("%s无版本信息数据"%registeNo)
    else:
        value = [[m["Result"][0]['RegisterNo'][4:17],m["Result"][0]['VersionNo'],m["Result"][0]['Name']]]
        write_excel_xls_append(book_name_xls, value)
        print(registeNo+"数据写入成功")